# LSTM

In [2]:
import yfinance as yf
import numpy as np
import pandas as pd

from pywt import dwt
from sklearn.preprocessing import MinMaxScaler
from keras.layers import LSTM, Dense
from keras.models import Sequential
from sklearn.metrics import mean_absolute_error

# Collect Apple stock price data
apple = yf.Ticker("AAPL")
data = apple.history(period="max")

# Preprocess the data
data.dropna(inplace=True)
data = data[['Close']]
scaler = MinMaxScaler(feature_range=(0, 1))
data = scaler.fit_transform(data)

# Decompose the data using wavelet transformation
coeffs = dwt(data, 'db4')
cA, cD = coeffs

# Smooth the data using moving average
window_size = 10
cA_smooth = np.zeros_like(cA)
for i in range(window_size, len(cA)):
    cA_smooth[i] = np.mean(cA[i-window_size:i])

# Build the LSTM model
model = Sequential()
model.add(LSTM(50, input_shape=(1, 1)))
model.add(Dense(1))
model.compile(loss='mae', optimizer='adam')

# Split the data into train and test sets
train_size = int(len(cA_smooth) * 0.8)
train, test = cA_smooth[:train_size], cA_smooth[train_size:]



In [4]:
train

array([[0.        , 0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        ],
       ...,
       [0.17032065, 0.17032065, 0.17032065, 0.17032065],
       [0.17138903, 0.17138903, 0.17138903, 0.17138903],
       [0.17254149, 0.17254149, 0.17254149, 0.17254149]])

In [ ]:
# Reshape the data for input into the LSTM model
train = train.reshape(train.shape[0], 1, 1)
test = test.reshape(test.shape[0], 1, 1)

# Train the LSTM model
model.fit(train, train, epochs=50, batch_size=1, verbose=2)

# Make predictions on the test set
predictions = model.predict(test)

# Evaluate the model using mean absolute error
mae = mean_absolute_error(test, predictions)
print("Mean Absolute Error:", mae)

# TCN (CNN)

In [25]:
import yfinance as yf
import numpy as np
import pandas as pd

from pywt import dwt
from sklearn.preprocessing import MinMaxScaler
from keras.layers import Conv1D, Dense, Dropout
from keras.models import Sequential
from sklearn.metrics import mean_absolute_error

# Collect Apple stock price data
apple = yf.Ticker("AAPL")
data = apple.history(period="max")

# Preprocess the data
data.dropna(inplace=True)
data = data[['Close']]
scaler = MinMaxScaler(feature_range=(0, 1))
data = scaler.fit_transform(data)

# Decompose the data using wavelet transformation
coeffs = dwt(data, 'db4')
cA, cD = coeffs

# Build the TCN model
model = Sequential()
# model.add(Conv1D(filters=64, kernel_size=2, activation='relu', input_shape=(4, 1)))
# model.add(Dropout(0.5))
# model.add(Conv1D(filters=32, kernel_size=4, activation='relu', dilation_rate=8))
# model.add(Dropout(0.5))
# model.add(Conv1D(filters=16, kernel_size=8, activation='relu', dilation_rate=4))
# model.add(Dropout(0.5))
# model.add(Conv1D(filters=8, kernel_size=8, activation='relu', dilation_rate=2))
# model.add(Dropout(0.5))
# model.add(Dense(1))

# model.add(InputLayer((4, 1)))
# model.add(LSTM(units = 64, return_sequences = True))
# model.add(LSTM(units = 32, return_sequences = True))
model.add(Conv1D(filters=32, kernel_size=2, activation='relu'))
for dilation_rate in (1,2,4,8,16,23):
    model.add(Conv1D(filters=32,kernel_size=2,dilation_rate=dilation_rate,strides=1,padding='causal',activation='relu'))
model.add(Conv1D(filters=1,kernel_size=1)) 
model.add(Dropout(0.2))
model.add(Dense(units = 1))
model.compile(loss='mae', optimizer='adam')

# Split the data into train and test sets
train_size = int(len(cA) * 0.8)
train, test = cA[:train_size], cA[train_size:]

# Reshape the data for input into the TCN model
train = train.reshape(train.shape[0], train.shape[1], 1)
test = test.reshape(test.shape[0], test.shape[1], 1)

# Train the TCN model
model.fit(train, train, epochs=50, batch_size=1, verbose=2)

# Make predictions on the test set
predictions = model.predict(test)

# Evaluate the model using mean absolute error
mae = mean_absolute_error(test, predictions)
print("Mean Absolute Error:", mae)

Epoch 1/50


ValueError: in user code:

    File "/home/baptiste/myworkdir/trading-sandbox/.venv/lib/python3.10/site-packages/keras/engine/training.py", line 1249, in train_function  *
        return step_function(self, iterator)
    File "/home/baptiste/myworkdir/trading-sandbox/.venv/lib/python3.10/site-packages/keras/engine/training.py", line 1233, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/home/baptiste/myworkdir/trading-sandbox/.venv/lib/python3.10/site-packages/keras/engine/training.py", line 1222, in run_step  **
        outputs = model.train_step(data)
    File "/home/baptiste/myworkdir/trading-sandbox/.venv/lib/python3.10/site-packages/keras/engine/training.py", line 1024, in train_step
        loss = self.compute_loss(x, y, y_pred, sample_weight)
    File "/home/baptiste/myworkdir/trading-sandbox/.venv/lib/python3.10/site-packages/keras/engine/training.py", line 1082, in compute_loss
        return self.compiled_loss(
    File "

In [26]:
train

array([[[0.00048199],
        [0.00048199],
        [0.00048199],
        [0.00048199]],

       [[0.00044126],
        [0.00044126],
        [0.00044126],
        [0.00044126]],

       [[0.00038695],
        [0.00038695],
        [0.00038695],
        [0.00038695]],

       ...,

       [[0.17586222],
        [0.17586222],
        [0.17586222],
        [0.17586222]],

       [[0.1771583 ],
        [0.1771583 ],
        [0.1771583 ],
        [0.1771583 ]],

       [[0.17754362],
        [0.17754362],
        [0.17754362],
        [0.17754362]]])

In [21]:
train[0]

array([[0.00048199],
       [0.00048199],
       [0.00048199],
       [0.00048199]])